In [14]:
from braindecode.util import set_random_seeds, np_to_var, var_to_np
import matplotlib.pyplot as plt
from moabb.datasets import BNCI2014001, Cho2017, PhysionetMI
from moabb.paradigms import MotorImagery
import numpy as np
from numpy.random import RandomState
import pickle
import time
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

import mne

import start_kits.util.shallow_net
from start_kits.util.utilfunc import get_balanced_batches
from start_kits.util.preproc import plot_confusion_matrix

from braindecode.models import ShallowFBCSPNet, EEGNetv1, EEGNetv4

cuda = torch.cuda.is_available()
print('gpu: ', cuda)
device = 'cuda' if cuda else 'cpu'

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
rng = RandomState(seed)

gpu:  True


In [15]:
import numpy as np
x_val = np.load('/home/ducanh/hain/hungp/NeurIPS_BEETL/numpy_data_binary/X_val.npy')

# original shape: (num_trial, num_electrons, num_samples) with each trial is 1 label
input = x_val[:2, :, :]
input = input.astype('float32') * 1e3
print(input.shape)

# add batch dims
input = np.expand_dims(input, axis=0)

# convert type to torch tensor
input = torch.from_numpy(input)
# reshape input size
input = input.permute(1, 2, 3, 0)

print(input.shape)

(2, 32, 256)
torch.Size([2, 32, 256, 1])


In [16]:
in_chans = 32

model = EEGNetv4(
    in_chans = in_chans,
    n_classes = 3,
    input_window_samples=256,
)

'''
TODO: Load model pretrained here
'''
checkpoint = torch.load('three_classes.pth')
model.load_state_dict(checkpoint['model_state_dict'])

if cuda:
    input = input.cuda()
    model = model.cuda()
model.eval()

EEGNetv4(
  (ensuredims): Ensure4d()
  (dimshuffle): Expression(expression=_transpose_to_b_1_c_0) 
  (conv_temporal): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=(0, 32), bias=False)
  (bnorm_temporal): BatchNorm2d(8, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (conv_spatial): Conv2dWithConstraint(8, 16, kernel_size=(32, 1), stride=(1, 1), groups=8, bias=False)
  (bnorm_1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (elu_1): Expression(expression=elu) 
  (pool_1): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
  (drop_1): Dropout(p=0.25, inplace=False)
  (conv_separable_depth): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), padding=(0, 8), groups=16, bias=False)
  (conv_separable_point): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bnorm_2): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (elu_2): Expression(expression=elu) 
  (pool_2): AvgPool

In [17]:
# with torch.no_grad:
out = model(input)
out.shape

torch.Size([2, 3])

In [18]:
out

tensor([[-0.9576, -1.0947, -1.2675],
        [-0.9187, -1.2136, -1.1913]], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [19]:
output = F.softmax(out, dim=1)
print(output)

tensor([[0.3838, 0.3347, 0.2815],
        [0.3990, 0.2971, 0.3038]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [20]:
# import pickle
# weight = pickle.load(open('pickle_weight.pkl','rb'))